# Cortexハンズオン Part.2 
    
レビューデータの分析準備を行います。以下の処理が行います：
    
### 1. テーブル作成と初期設定
* 分析用テーブル (CUSTOMER_ANALYSIS) の作成
    
### 2. レビューテキスト処理
* **テキスト分割**: SPLIT_TEXT_RECURSIVE_CHARACTER関数を使用して、レビューテキストを300文字以内のチャンクに分割
* **翻訳処理**: TRANSLATE関数を使用して、日本語のテキストを英語に翻訳
* **感情分析**: SENTIMENT関数を使用して、翻訳されたテキストの感情スコア (-1〜1) を算出
* **ベクトル化**: EMBED_TEXT_1024関数を使用して、テキストを1024次元のベクトルデータに変換
### 3. 分析用データ生成
* **カテゴリ抽出**: CLASSIFY_TEXT関数を使用して各レビューの内容を分析し、事前に定義されたカテゴリから最も関連性の高いものを設定
* **単語の抽出**: COMPLETE関数の構造化出力機能を使用して各レビューから重要な単語 (名詞、動詞、形容詞) を抽出し、 出現回数をカウント

### 4. Cortex Search Service作成
### 5. Cortex Analystで利用するセマンティックモデルの確認

In [ ]:
CREATE TABLE IF NOT EXISTS CUSTOMER_ANALYSIS (
    analysis_id NUMBER AUTOINCREMENT,
    review_id VARCHAR(20),
    product_id VARCHAR(10),
    customer_id VARCHAR(10),
    rating NUMBER(2,1),
    review_text TEXT,
    review_date TIMESTAMP_NTZ,
    purchase_channel VARCHAR(20),
    helpful_votes NUMBER(5),
    chunked_text TEXT,
    embedding VECTOR(FLOAT, 1024),
    sentiment_score FLOAT,
    updated_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
);

### 2. レビューテキスト処理
1. 未処理のレビューテキストを取得

In [ ]:
SELECT r.*
FROM CUSTOMER_REVIEWS r
LEFT JOIN CUSTOMER_ANALYSIS a
ON r.review_id = a.review_id
WHERE a.review_id IS NULL

2. 文章全体を英語に翻訳

### TRANSLATE関数
```sql
SELECT  
  SNOWFLKE.CORTEX.TRANSLATE(
    'こんにちは。Es ist heute schönes Wetter.', '', 'en'
  );
```
```
Hello. It's a beautiful day today.
```
* シンプルな関数で14ヶ国語の翻訳をすることが可能
* 翻訳元の言語は省略することができ、省略した際には翻訳元言語を自動的に認識
* 翻訳元の言語に複数の言語が混在している場合でも翻訳先言語に翻訳することが可能

In [ ]:
SELECT *, SNOWFLAKE.CORTEX.TRANSLATE(REVIEW_TEXT, '', 'en') as review_text_en FROM {{non_reviewed_result}}

3. 翻訳されたテキストを感情分析

### SENTIMENT関数
```sql
SELECT  
  SNOWFLKE.CORTEX.SENTIMENT(
    'This product was borken so please return it.'
  );
```
```
-0.83841026.
```
* テキストから感情スコアを `[-1 〜 1]` の範囲で算出する
* 例えばカスタマーレビューやアンケートなどから統一した基準で感情をスコアリング可能
* 現状は英語のテキストのみ対応しているため、前段にTRANSLATE関数を実施するなどの工夫が必要

In [ ]:
SELECT * EXCLUDE review_text_en, SNOWFLAKE.CORTEX.SENTIMENT(review_text_en) as score from {{translate_review}}

4. テキストをチャンクに分割

### SPLIT_TEXT_RECURSIVE_CHARACTER関数
```sql
SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
  text: '### Heading 1\\n Sample Title ...’,
  format: 'markdown’,
  chunk size: 50,
  overlap: 10,
)
```
* 使いやすいSQL関数で`長いドキュメントをより効率的な検索のために小さな部分に分割`
* markdownモードを使用してドキュメント構造を保持し、ヘッダー、コードブロック、テーブルなどのセクションを整理された`自己完結型のユニットに分割`
* チャンクサイズとオーバーラップ設定を簡単に調整して、`複数のチャンク分割戦略を素早く試験`

In [ ]:
SELECT 
a.*,
c.value::varchar as chunk
FROM {{sentiment_score}} a,
LATERAL FLATTEN (input =>
    SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
        review_text,
        'none',
        300,
        30
)) as c

5. ベクトル化してテーブルに登録

 ### Cortex Embedding関数
```sql
SELECT
  SNOWFLAKE.CORTEX.EMBED_TEXT_1024(
    'multilingual-e5-large',
    product_name
  ) as product_name_embed
FROM product_master
```
- 対応しているモデル

| embed_text_768 | embed_text_1024 |
| ---------- | ---------- |
| snowflake-arctic-embed-m-v1.5 | snowflake-arctic-embed-l-v2.0
| snowflake-arctic-embed-m | nv-embed-qa-4
| e5-base-v2 | multilingual-e5-large
| | voyage-multilingual-2 |

日本語精度の観点だと、下記の順番で良い

`voyage-multi > multi-e5-large > snowflake-arctic`

In [ ]:
import streamlit as st
EMBEDDING_MODELS = [
    "multilingual-e5-large",
    "voyage-multilingual-2",
    "snowflake-arctic-embed-l-v2.0",
    "nv-embed-qa-4"
]
embedding_model = st.selectbox(
    "Embeddingモデルを選択してください",
    EMBEDDING_MODELS,
    index=0
)

In [ ]:
INSERT INTO CUSTOMER_ANALYSIS (
    review_id,
    product_id,
    customer_id,
    rating,
    review_text,
    review_date,
    purchase_channel,
    helpful_votes,
    chunked_text,
    embedding,
    sentiment_score
)
SELECT
    review_id,
    product_id,
    customer_id,
    rating,
    review_text,
    review_date,
    purchase_channel,
    helpful_votes,
    chunk,
    SNOWFLAKE.CORTEX.EMBED_TEXT_1024('{{embedding_model}}', chunk),
    score
FROM {{chunk_result}}

6. 最終結果を確認

In [ ]:
SELECT * FROM CUSTOMER_ANALYSIS;

### 3. 分析用データ生成
**カテゴリ抽出**: CLASSIFY_TEXT関数を使用して各レビューの内容を分析し、事前に定義されたカテゴリから最も関連性の高いものを設定

0. レビュー管理用テーブルの作成

In [ ]:
-- カテゴリマスタ
CREATE TABLE IF NOT EXISTS REVIEW_CATEGORIES (
            category_id NUMBER AUTOINCREMENT,
            category_name VARCHAR(100),
            description TEXT,
            created_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
            updated_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
);

In [ ]:
-- レビュータグテーブル
CREATE TABLE IF NOT EXISTS REVIEW_TAGS (
    tag_id NUMBER AUTOINCREMENT,
    review_id VARCHAR(20),
    category_name VARCHAR(100),
    confidence_score FLOAT,
    created_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
    updated_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
)

In [ ]:
-- 重要単語テーブル
CREATE TABLE IF NOT EXISTS REVIEW_WORDS (
    word_id NUMBER AUTOINCREMENT,
    review_id VARCHAR(20),
    word VARCHAR(100),
    word_type VARCHAR(50),
    frequency NUMBER,
    created_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
    updated_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
)

In [ ]:
DEFAULT_CATEGORIES = [
    "商品の品質",
    "価格",
    "接客サービス",
    "店舗環境",
    "配送・梱包",
    "品揃え",
    "使いやすさ",
    "鮮度",
    "その他"
]
categories_values = ", ".join([f"('{category}')" for category in DEFAULT_CATEGORIES])

In [ ]:
-- カテゴリの初期値登録
INSERT INTO REVIEW_CATEGORIES (category_name)
SELECT category FROM (VALUES {{categories_values}}) AS v(category)
WHERE category NOT IN (SELECT category_name FROM REVIEW_CATEGORIES)

1. 未レビューデータの抽出

In [ ]:
SELECT r.*
FROM CUSTOMER_REVIEWS r
LEFT JOIN REVIEW_TAGS t
ON r.review_id = t.review_id
WHERE t.review_id IS NULL

2. CLASSIFY_TEXT関数を使用して各レビューの内容を分析し、事前に定義されたカテゴリから最も関連性の高いものを設定

### CLASSIFY_TEXT 関数
```sql
SNOWFLAKE.CORTEX.CLASSIFY_TEXT (
  review_text, -- 対象文字列
  categories, -- 分類で使用するカテゴリ
)
```

インプットテキストの内容をユーザが指定したカテゴリにLLMが分類

In [ ]:
SELECT
    review_id,
    SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
        review_text,
        {{DEFAULT_CATEGORIES}},
        {
            'task_description': 'レビューテキストの内容から最も適切なカテゴリを選択してください。'
        }
        
    ) as classification
FROM {{non_tags_result}}


3. タグ情報を登録

In [ ]:
INSERT INTO REVIEW_TAGS (
    review_id,
    category_name,
    confidence_score
)
SELECT
  review_id,
  classification:label::varchar,
  1.0
FROM {{classified_result}}

In [ ]:
-- 登録結果確認
SELECT * FROM REVIEW_TAGS;

**単語の抽出**: COMPLETE関数の構造化出力機能を使用して各レビューから重要な単語 (名詞、動詞、形容詞) を抽出し、 出現回数をカウント

In [ ]:
-- truncate table REVIEW_WORDS;

-- 未処理のレビューを取得
SELECT r.*
FROM CUSTOMER_REVIEWS r
LEFT JOIN REVIEW_WORDS w
ON r.review_id = w.review_id
WHERE w.review_id IS NULL

2. COMPLETE関数 (with 構造化出力) を利用して単語の頻出頻度を抽出する

### COMPLETE 関数 (with 構造化出力)
```sql
SELECT SNOWFLAKE.CORTEX.COMPLETE('mistral-large2', [
        {
        'role': 'user',
        'content': 'Return the customer sentiment for the following review: New kid on the block, this pizza joint! The pie arrived neither in a flash nor a snail\'s pace, but the taste? Divine! Like a symphony of Italian flavors, it was a party in my mouth. But alas, the party was a tad pricey for my humble abode\'s standards. A mixed bag, I\'d say!'
            }
    ],
    {
        'temperature': 0,
        'max_tokens': 1000,
        'response_format':{
            'type':'json',
            'schema':{'type' : 'object','properties' : {'sentiment_categories':{'type':'array','items':{'type':'object','properties':
            {'food_quality' : {'type' : 'string'},'food_taste': {'type':'string'}, 'wait_time': {'type':'string'}, 'food_cost': {'type':'string'}},'required':['food_quality','food_taste' ,'wait_time','food_cost']}}}}
            }
    }
);
```
response_format引数にJSONスキーマを指定する。指定されたJSONスキーマオブジェクトは、必須フィールドを含め、生成されるテキストが準拠しなければならない構造、データ型、制約を定義します。単純なタスクの場合、出力フォーマットの詳細を指定する必要はありませんし、「JSONで応答する 」ようにモデルに指示する必要もありません。より複雑なタスクの場合、モデルにJSONで応答するように指示することで、精度を向上させることができます。

In [ ]:
COMPLETE_MODELS = [
    "claude-3-5-sonnet",
    "deepseek-r1",
    "mistral-large2",
    "llama3.3-70b",
    "snowflake-llama-3.3-70b"
]
complete_model = st.selectbox(
    "Completeモデルを選択してください",
    COMPLETE_MODELS,
    index=0
)

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
                        '{{complete_model}}',  -- 使用するLLMモデル
                        [
                            {
                                'role': 'system',
                                'content': '複数のレビューテキストから重要な単語を抽出し、品詞と出現回数を分析してください。各レビューごとに分析結果を提供してください。'
                            },
                            {
                                'role': 'user',
                                'content': TO_JSON(OBJECT_CONSTRUCT('id', review_id, 'text', review_text))  -- 分析する複数レビューテキスト（JSONフォーマット）
                            }
                        ],
                        {
                            'temperature': 0,  -- 生成結果の多様性（0=決定的な出力）
                            'max_tokens': 2000,  -- 最大応答トークン数を増やす
                            'response_format': {
                                'type': 'json',
                                'schema': {
                                    'type': 'object',
                                    'properties': {
                                        'reviews_analysis': {
                                            'type': 'array',
                                            'items': {
                                                'type': 'object',
                                                'properties': {
                                                    'review_id': {
                                                        'type': 'string',
                                                        'description': 'レビューのID'
                                                    },
                                                    'words': {
                                                        'type': 'array',
                                                        'items': {
                                                            'type': 'object',
                                                            'properties': {
                                                                'word': {
                                                                    'type': 'string',
                                                                    'description': '抽出された単語'
                                                                },
                                                                'type': {
                                                                    'type': 'string',
                                                                    'enum': ['名詞', '動詞', '形容詞'],
                                                                    'description': '品詞（名詞、動詞、形容詞のいずれか）'
                                                                },
                                                                'frequency': {
                                                                    'type': 'integer',
                                                                    'description': '単語の出現回数'
                                                                }
                                                            },
                                                            'required': ['word', 'type', 'frequency']
                                                        }
                                                    }
                                                },
                                                'required': ['review_id', 'words']
                                            }
                                        }
                                    },
                                    'required': ['reviews_analysis']
                                }
                            }
                        }
                    ) as result
FROM {{non_word_reviews}}

3. 構造化されている出力結果から単語情報を処理します

In [ ]:
import json
import pandas as pd

review_words_list = []

# 各レビューの単語情報を処理
for item in word_extracts.to_pandas().itertuples(index=False):
    result = json.loads(item[0])

    structured_output = result.get('structured_output', [{}])[0].get('raw_message', {})
    reviews_data = structured_output.get('reviews_analysis', [])

    for review_data in reviews_data:
        review_id = review_data.get('review_id')
        words_data = review_data.get('words', [])            

        # 単語情報をテーブルに保存
        local_words_count = 0
        for word in words_data:
            # 単語データの検証
            if not all(k in word for k in ['word', 'type', 'frequency']):
                continue

            review_words_list.append({
                "REVIEW_ID": review_id,
                "WORD": word['word'],
                "WORD_TYPE": word['type'],
                "FREQUENCY": word['frequency'],
            })

word_df = pd.DataFrame(review_words_list)
session.write_pandas(word_df, "REVIEW_WORDS", auto_create_table=False)

### 4. Cortex Search Service作成

In [ ]:
-- 1. 使用するデータベースとスキーマを指定
USE DATABASE SNOWRETAIL_DB;
USE SCHEMA SNOWRETAIL_SCHEMA;

-- 2. Cortex Search Serviceを作成
CREATE OR REPLACE CORTEX SEARCH SERVICE snow_retail_search_service
    ON content                        -- 検索対象のテキストデータが含まれるカラム
    ATTRIBUTES title, document_type, department -- 検索結果に含めたい属性カラム
    WAREHOUSE = 'COMPUTE_WH' -- サービスのインデックス作成に使用するウェアハウス
    TARGET_LAG = '1 day'              -- データソーステーブルの変更がインデックスに反映されるまでの最大遅延時間
    EMBEDDING_MODEL = 'voyage-multilingual-2' -- テキストのベクトル化に使用する埋め込みモデル 
    AS
        -- 検索対象とするデータを定義するSELECT文
        SELECT
            document_id,  -- 各ドキュメントの一意なID
            title,        -- 属性として使用するタイトル
            content,      -- 検索対象のテキストコンテンツ
            document_type,-- 属性として使用するドキュメントタイプ
            department,   -- 属性として使用する部署名
            created_at,   -- オプション：メタデータとして含める作成日時
            updated_at,   -- オプション：メタデータとして含める更新日時
            version       -- オプション：メタデータとして含めるバージョン
        FROM SNOW_RETAIL_DOCUMENTS; -- データソースとなるテーブル

### 5. Cortex Analystで利用するセマンティックモデルの確認
Cortes Analystで利用するYamlファイルの内容

In [ ]:
from snowflake.snowpark.context import get_active_session
import streamlit as st
import sys

session = get_active_session()
yml = session.file.get("@SNOWRETAIL_DB.SNOWRETAIL_SCHEMA.SEMANTIC_MODEL_STAGE/sales_analysis_model.yaml","./")

import yaml

with open('sales_analysis_model.yaml', encoding='utf-8') as f:
    loaded_document = yaml.safe_load(f)

st.code(yaml.dump(loaded_document, allow_unicode=True), "yaml")